In [1]:
import numpy as np
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, HTML
from IPython.display import display, IFrame
from pyscal_rdf.structure import StructureGraph

import pyscal_rdf_gui.plot as pgp
import pyscal_rdf_gui.create as pgc
from pyscal_rdf_gui.create_structure import CreateStructure
from pyscal_rdf_gui.create_gb import CreateGB
from pyscal_rdf_gui.create_vacancy import CreateVacancy
from pyscal_rdf_gui.read_structure import ReadStructure
from pyscal_rdf_gui.write_structure import WriteStructure
from pyscal_rdf_gui.sparql_query import SparqlQuery
from pyscal_rdf_gui.auto_query import AutoQuery
from pyscal_rdf_gui.ontology import Ontology
from pyscal_rdf_gui.properties import dataprops

from pyscal3.grain_boundary import GrainBoundary

In [2]:
from pyscal_rdf_gui.image_elements import image_widget

In [3]:
image_widget

HTML(value='\n<img src="https://datashare.mpcdf.mpg.de/index.php/apps/files_sharing/ajax/publicpreview.php?x=1…

In [4]:
#this has to be created first to enable status changes
gb = CreateGB()
vac = CreateVacancy()
species = None

In [5]:
#Create a general element viewer
atom_viewer = pgc.output()

In [6]:
#Structure creation widgets
st = CreateStructure(theme="teal")

def create_structure(b=None):
    """
    Function to create structure
    """
    structure = st.dropdown.value
    add_to_graph = st.checkbox.value
    rep = st.repetition_box.value
    lattice_parameter = st.lattice_parameter_box.value
    if structure in ["l12", "b2"]:
        element = [st.element_box.value, st.element_box_2.value]
    else:
        element = st.element_box.value
    global sys
    sys = g.create_structure(structure, 
                             lattice_constant=lattice_parameter,
                             repetitions=[rep, rep, rep],
                             add_to_graph=add_to_graph,
                             element=element,
                             names=True)
    global species
    species = element
    pgp.system(sys, atom_viewer)
    create_count_text()
    return sys

def update_element_fields(change):
    if st.dropdown.value in ["l12", "b2"]:
        st.element_box_2.disabled = False
    else:
        st.element_box_2.disabled = True
    
st.dropdown.observe(update_element_fields)

@st.run_button.on_click
def structure_plot_on_click(b):
    sys = create_structure()
    #turn on gb creation
    gb.run_button.disabled = False
    vac.run_button.disabled = False

In [7]:
defect_dropdown = pgc.dropdown('Add defect', ['None', 'grain boundary', 
                            'vacancy'], 
                              value='None')

defect_text = "Add a defect"
defect_header = pgc.header(defect_text, theme="teal")
output_defect = pgc.output()

def show_defect_menu(change):
    if defect_dropdown.value == "grain boundary":
        output_defect.clear_output()        
        with output_defect:
            display(gb.panel)
    elif defect_dropdown.value == "vacancy":
        output_defect.clear_output()        
        with output_defect:
            display(vac.panel)
        

defect_dropdown.observe(show_defect_menu)

defect_panel = VBox(children=[defect_header,
                             defect_dropdown,
                             output_defect])

@gb.sigma_button.on_click
def show_sigmas(b):
    axis_box = [int(x) for x in gb.axis_box.value.split()]
    gbc = GrainBoundary()
    gbc.axis = axis_box
    gbc.limit = 10
    sigmas = gbc.get_possible_sigma()[1:]
    gb.sigma_dropdown = pgc.dropdown("Sigma",
                                 sigmas,)
    #gb.widget_list.append(sigma_dropdown)
    gb.sigma_hb = HBox(children=[gb.sigma_dropdown, gb.gb_plane_button])
    output_defect.clear_output()        
    with output_defect:
        vbox = VBox(children=[gb.panel, gb.sigma_hb])
        display(vbox)      

#now using sigma, we need to create a dropdown of possible Gb planes
@gb.gb_plane_button.on_click
def show_gb_planes(b):
    axis_box = [int(x) for x in gb.axis_box.value.split()]
    gbc = GrainBoundary()
    gbc.axis = axis_box
    gbc.limit = 10
    gbc.gb_index_limit = 3
    gbc.sigma = gb.sigma_dropdown.value
    _ = gbc.calculate_theta()
    planes, _, _, _ = gbc.calculate_possible_gb_planes()
    planes = [" ".join(np.array(plane).astype(str)) for plane in planes]
    gb.gb_plane_dropdown = pgc.dropdown("GB Plane",
                                    planes,)
    gb.gb_plane_hb = VBox(children=[gb.rep_box,
                                    HBox(children=[gb.gb_plane_dropdown, gb.run_button])])
    output_defect.clear_output()        
    with output_defect:
        vbox = VBox(children=[gb.panel, gb.sigma_hb, gb.gb_plane_hb])
        display(vbox)    
    

@gb.run_button.on_click
def gb_plot_on_click(b):
    #turn on gb creation
    #convert to arrays
    axis_box = [int(x) for x in gb.axis_box.value.split()]
    gb_plane_box = [int(x) for x in gb.gb_plane_box.value.split()]
    global sys
    sys = g.create_grain_boundary(axis_box, gb.sigma_dropdown.value, 
                            gb_plane_box, element=species,
                            repetitions=[int(x) for x in gb.rep_box.value.split()],
                            lattice_constant=sys.atoms._lattice_constant,
                            names=True)
    pgp.system(sys, atom_viewer)
    gb.run_button.disabled = True
    output_defect.clear_output()
    defect_dropdown.value = "None"
    create_count_text()
    return sys

@vac.run_button.on_click
def vac_plot_on_click(b):
    global sys
    g.add_structure_to_graph(sys)
    value_type = vac.vacancy_dropdown.value
    value = float(vac.vacancy_box.value)
    if value_type == "concentration":
        if (value >= 1) or (value <= 0):
            output_defect.clear_output()
            with output_defect:
                print("Vacancy concentration should be between 0-1")
        else:
            number = int(value*sys.natoms)
            if number < 1:
                output_defect.clear_output()
                with output_defect:
                    print("At least one atom should be deleted")
    else:
        if (value <=0) or (value >= sys.natoms):
            output_defect.clear_output()
            with output_defect:
                print(f'Atom number should be between 1-{sys.natoms-1}')
        else:
            number = value
    #choose randomly 
    tbd = np.random.randint(sys.natoms, size=int(number))
    sys.delete(indices=list(tbd))
    c = (number/sys.natoms)
    g.add_vacancy(c)
    pgp.system(sys, atom_viewer)
    vac.run_button.disabled = True
    output_defect.clear_output()
    defect_dropdown.value = "None"
    create_count_text()
    return sys    

In [8]:
#Read structure widgets
rs = ReadStructure(theme="teal")

def read_structure(b=None):
    #write file
    with open("temp", "wb") as fout:
        fout.write(rs.upload.value[0]['content'])
    global sys
    sys = g.read_structure("temp", 
                           format=rs.dropdown.value)
    pgp.system(sys, atom_viewer)
    create_count_text()
    return sys    
    
@rs.run_button.on_click
def process_file_on_click(b):
    sys = read_structure()

In [9]:
#write structure widgets
ws = WriteStructure(theme="teal")

def write_structure(b=None):
    outfile = f'out.{ws.dropdown.value}'
    if ws.dropdown.value in ['turtle', 'json-ld', 'xml', 'n3']:
        sg = g.get_sample(g.sample)
        sg.write(outfile, format=ws.dropdown.value)
    else:
        sys.to_file(outfile, format=ws.dropdown.value)

@ws.run_button.on_click
def write_file_on_click(b):
    write_structure()
    ws.output.clear_output()
    with ws.output:
        display(pgc.download(f'out.{ws.dropdown.value}'))

In [10]:
#Creation panel
creation_panel = VBox(children=[
                      st.panel,
                      defect_panel,
                      rs.panel])


vis_label = pgc.header("Visualise structure", theme="teal")
vis_structure_panel = VBox(children=[vis_label,
                      atom_viewer,
                      ws.panel,
                      ])

creation_tab = HBox(children=[creation_panel, 
                              vis_structure_panel])

In [11]:
#Widgets for querying
sq = SparqlQuery()

@sq.run_button.on_click
def query_plot_on_click(b):
    run_query()
    
def run_query(b=None):
    query = sq.query_box.value
    res = g.graph.query(query)
    sq.output.clear_output()
    with sq.output:
        for r in res:
            print(r[0])

In [12]:
#Ontology visualisation widgets
on = Ontology()
on.output.clear_output()
with on.output:
    display(IFrame(src="https://service.tib.eu/webvowl/#iri=http://purls.helmholtz-metadaten.de/cmso/", width='100%', height='500px'))

In [13]:
wso = WriteStructure(theme="teal")

def write_structure_onto(b=None):
    outfile = f'out.{ws.dropdown.value}'
    if ws.dropdown.value in ['turtle', 'json-ld', 'xml', 'n3']:
        sg = g.get_sample(g.sample)
        sg.write(outfile, format=ws.dropdown.value)
    else:
        sys.to_file(outfile, format=ws.dropdown.value)
        
@wso.run_button.on_click
def write_file_on_click(b):
    write_structure_onto()
    wso.output.clear_output()
    with wso.output:
        display(pgc.download(f'out.{wso.dropdown.value}'))

In [14]:
#Automated query code
aq = AutoQuery()

def update_compare_output(change):
    aq.compare_output.clear_output()
    if aq.property_dropdown.value in dataprops.keys():
        with aq.compare_output:
            display(aq.compare_options)

with aq.compare_output:
    display(aq.compare_options)
    
aq.property_dropdown.observe(update_compare_output)

def update_input_fields(change):
    aq.input_output.clear_output()
    if aq.compare_options.value == " equal to ":
        equal_inp = HBox(children=[aq.input_field_1])
        with aq.input_output:
            display(equal_inp)
    elif aq.compare_options.value == " in between ":
        equal_inp = HBox(children=[aq.input_field_1, 
                                   pgc.text("and"), 
                                   aq.input_field_2])
        with aq.input_output:
            display(equal_inp)
    elif aq.compare_options.value == " equal to either ":
        equal_inp = HBox(children=[aq.input_field_1, 
                                   pgc.text("or"), 
                                   aq.input_field_2])
        with aq.input_output:
            display(equal_inp)

with aq.input_output:
    display(HBox(children=[aq.input_field_1]))
    
aq.compare_options.observe(update_input_fields)

def convert_to_datatype(value, dtype):
    if dtype == "string":
        return str(value)
    elif dtype == "integer":
        return int(value)
    else:
        return float(value)
    
@aq.run_button.on_click
def run_query_on_click(b):
    run_auto_query()
    
def run_auto_query(b=None):
    #if it is equal to
    dtype = dataprops[aq.property_dropdown.value]["dtype"]
    if aq.compare_options.value == " equal to ":
        value = convert_to_datatype(aq.input_field_1.value, dtype)
    elif aq.compare_options.value == " in between ":
        value = [convert_to_datatype(aq.input_field_1.value, dtype),
                 convert_to_datatype(aq.input_field_2.value, dtype)]
    else:
        value = [convert_to_datatype(aq.input_field_1.value, "string"),
                 convert_to_datatype(aq.input_field_2.value, "string")]
    
    results = g.query_sample(dataprops[aq.property_dropdown.value]["onto"],
              value, 
              return_query=False)
    results = [r[0] for r in results]
    global result_dropdown
    result_dropdown = pgc.dropdown("Samples", results)
    disp = HBox(children=[result_dropdown,
                         aq.plot_structure_button,
                         aq.visualise_graph_button,
                         wso.dropdown,
                         wso.run_button,
                         wso.output])
    aq.result_output.clear_output()
    with aq.result_output:
        display(disp)
        
@aq.show_button.on_click
def run_show_on_click(b):
    run_auto_query_show()
    
def run_auto_query_show(b=None):
    #if it is equal to
    dtype = dataprops[aq.property_dropdown.value]["dtype"]
    if aq.compare_options.value == " equal to ":
        value = convert_to_datatype(aq.input_field_1.value, dtype)
    elif aq.compare_options.value == " in between ":
        value = [convert_to_datatype(aq.input_field_1.value, dtype),
                 convert_to_datatype(aq.input_field_2.value, dtype)]
    else:
        value = [convert_to_datatype(aq.input_field_1.value, "string"),
                 convert_to_datatype(aq.input_field_2.value, "string")]
    
    results, query = g.query_sample(dataprops[aq.property_dropdown.value]["onto"],
              value, 
              return_query=True)
    aq.result_output.clear_output()
    with aq.result_output:
        display(query)



@aq.plot_structure_button.on_click
def plot_structure_on_click(b):
    run_plot_structure()

def run_plot_structure(b=None):
    #get sys from sample
    sys = g.get_system_from_sample(result_dropdown.value) 
    pgp.system(sys, aq.result_plotter)
    
@aq.visualise_graph_button.on_click
def visualise_graph_on_click(b):
    run_visualise_graph()

def run_visualise_graph(b=None):
    #get sys from sample
    sgraph, na = g.get_sample(result_dropdown.value, 
                              no_atoms=True)
    aq.result_plotter.clear_output()
    dot = sgraph.visualise(backend="graphviz")
    with aq.result_plotter:
        display(dot)
  

In [15]:
#Create a structure graph
g = StructureGraph()

In [16]:
#widget to show number of samples
#this has to come after the main graph is created
count_output = pgc.output()

def create_count_text():
    text = "Database has %d samples"%g.n_samples
    onto_label = pgc.text(text)
    count_output.clear_output()
    with count_output:
        display(onto_label)

create_count_text()    

In [17]:
authors_panel = widgets.HTML("""
<b><font color='#006064'>IUC17: Ontologies for defects in crystals</b><br>
<font color='#006064'> TA-OMS: Abril Guzmán, Volker Hofmann, Stefan Sandfeld <br> TA-WSD: Sarath Menon, Tilmann Hickel <br>
<b><font color='#006064'> Resources </b><br>
<a href=\"https://github.com/Materials-Data-Science-and-Informatics/cmso-ontology\">CMSO Ontology</a> <br> 
<a href=\"https://github.com/pyscal/pyscal_rdf\">pyscal-rdf</a>
             """)

In [18]:
image_panel = widgets.HTML("""
<img src="https://datashare.mpcdf.mpg.de/index.php/apps/files_sharing/ajax/publicpreview.php?x=1920&y=619&a=true&file=Demo.png&t=qjr8s9QYEBE2gnb&scalingup=0" width="25%" align="justify">
<table table style='table-layout:fixed;width:100%' border="0">
  <th style='width: 65%;'> </th>
<tr>
  <td>
    <img src="https://www.mpie.de/4475264/original-1609193623.jpg?t=eyJ3aWR0aCI6MTIwMCwiaGVpZ2h0IjpudWxsLCJmaXQiOm51bGwsIm9ial9pZCI6NDQ3NTI2NH0%3D--73876e3c61ca1ef4ba1ca8817030fbd88c980e0b" width="25%">
    &nbsp;<img src="https://www.fz-juelich.de/de/ias/bilder/ias-9-emblem-5-3.png/@@images/image/teaser" width="12%">
    &nbsp;&nbsp;<img src="https://upload.wikimedia.org/wikipedia/de/thumb/8/8b/J%C3%BClich_fz_logo.svg/1200px-J%C3%BClich_fz_logo.svg.png" width="15%">
    &nbsp;&nbsp;&nbsp;&nbsp;<img src="https://www.nfdi.de/wp-content/uploads/2022/03/Logo_NFDI-MatWerk-1000px.png" width="10%">
  </td>
</tr>
</table>
""")

In [19]:
extra_panel = VBox(children=[authors_panel, image_panel])

In [20]:
#final tab arrangement
tab = widgets.Tab(children=[creation_tab,
                            aq.panel,
                            sq.panel,
                            on.panel,
                            extra_panel])
tab.set_title(0, 'Structure Creation')
tab.set_title(1, 'Automated Query')
tab.set_title(2, 'SPARQL Query')
tab.set_title(3, 'Ontology')
tab.set_title(4, 'About')


main = widgets.VBox(children=[tab,
                             count_output])
main